In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from preprocfromjannick import list_of_strings

In [3]:
list_of_strings()

[['absolutly loved iphone xr wa wonderful'], ['described “fully unlocked” received excellent condition except hour phone carrieramazon apple turn thing wa locked  thanks returned buying new one apple directly'], ['wa little sceptical buying phone review said phone received wa locked slight damage however one received wa perfect came fully unlocked  undamaged look baught straight apple far overall  star complaint'], ['took little arrive great phone nice picture pretty heavy though iphones im going back samsung'], ['purchased phone daughter came scratch mark however come headphone'], ['pésimo el teléfono vino bloqueado con clave con fotos de la persona anterior dentro del mismo'], ['world worst thing ever never get scam'], ['don’t know everyone’s giving seller bad review excellent phone fast shipping  star  back buy soon'], ['received iphone day ago everything working there’s barely noticeable scratch screen normal since refurbished wa hesitant buy due bad review phone unlocked checked a

[['absolutly loved iphone xr wa wonderful'],
 ['described “fully unlocked” received excellent condition except hour phone carrieramazon apple turn thing wa locked  thanks returned buying new one apple directly'],
 ['wa little sceptical buying phone review said phone received wa locked slight damage however one received wa perfect came fully unlocked  undamaged look baught straight apple far overall  star complaint'],
 ['took little arrive great phone nice picture pretty heavy though iphones im going back samsung'],
 ['purchased phone daughter came scratch mark however come headphone'],
 ['pésimo el teléfono vino bloqueado con clave con fotos de la persona anterior dentro del mismo'],
 ['world worst thing ever never get scam'],
 ['don’t know everyone’s giving seller bad review excellent phone fast shipping  star  back buy soon'],
 ['received iphone day ago everything working there’s barely noticeable scratch screen normal since refurbished wa hesitant buy due bad review phone unlocked c